<a href="https://colab.research.google.com/github/alexandreblima/IC-THS/blob/master/LeNet_CNN_mnist_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# arquivo: LeNet_CNN_mnist_tensorboard.py
# Adaptação dos códigos do livro "Deep Learning with TensorFlow2
# and Keras: Regression, Convnets, GANs RNNs, NLP and more with
# TensorFlow2 and the Keras API", de Antonio Gulli, Amita Kapoor 
# e Sujit Pal, 2a ed., Packt. 
# código para TensorFlow 2.2.0 Spyder - Anaconda
# LeNet CNN - base de dados MNIST

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import datetime
print(tf.__version__)

%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

# Classe que define a rede LeNet
class LeNet:
	@staticmethod
	def build(input_shape, classes):
		model = models.Sequential()
		# CONV => RELU => POOL
		model.add(layers.Convolution2D(20, (5, 5), activation='relu',
			input_shape=input_shape))
		model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		# CONV => RELU => POOL
		model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
		model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		# Flatten => RELU layers
		model.add(layers.Flatten())
		model.add(layers.Dense(500, activation='relu'))
		# a softmax classifier
		model.add(layers.Dense(classes, activation="softmax"))
		return model

# parâmetros da rede e do treinamento
EPOCHS = 15
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT=0.90

IMG_ROWS, IMG_COLS = 28, 28 # dimensões da imagem de entrada
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
NB_CLASSES = 10  # número de saídas = número de dígitos

# Carrega dados e rótulos
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

# reformata dados para o formato array do numpy
X_train = X_train.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

# normaliza dados
X_train, X_test = X_train / 255.0, X_test / 255.0

# representação numérica dos dados em ponto flutuante - 32 bits
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# one-hot encode (vetorização) dos rótulos
y_train = tf.keras.utils.to_categorical(y_train, NB_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NB_CLASSES)

# construção da rede
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

# compilação da rede
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])

# sumário da rede
model.summary()

# Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

# treinamento
treino = model.fit(X_train, y_train, 
		batch_size=BATCH_SIZE, epochs=EPOCHS, 
		verbose=VERBOSE, validation_split=VALIDATION_SPLIT,
		callbacks=tensorboard_callback)

treino_dic = treino.history 
# "treino.history" retorna um contêiner de dicionário (uma generalização do conceito de lista): tipo "dict" 
# Um dicionário contém pares de (chave, valor)
treino_dic.keys()
# chaves no tf 2.1.0: ['loss', 'accuracy', 'val_loss', 'val_accuracy']

import matplotlib.pyplot as plt

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

acc = treino.history['accuracy']
val_acc = treino.history['val_accuracy']
loss = treino.history['loss']
val_loss = treino.history['val_loss']

epochs = range(len(acc))

plt.clf()

plt.plot(epochs, acc, 'b', label='Acurácia de treinamento')
plt.plot(epochs, val_acc, 'r', label='Acurácia de validação')
plt.title('Curvas de aprendizado', fontdict=font)
plt.xlabel('tempo (épocas)', fontdict=font)
plt.ylabel('Acurácia', fontdict=font)
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Erro de treinamento')
plt.plot(epochs, val_loss, 'r', label='Erro de validação')
plt.title('Curvas de aprendizado ', fontdict=font)
plt.xlabel('tempo (épocas)', fontdict=font)
plt.ylabel('Erro', fontdict=font)
plt.legend()

plt.show()

# resultados: custo (loss) e acurácia - TESTE 
score = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

%tensorboard --logdir logs/fit

# New Section